In [ ]:
import os
import pickle
import joblib
import numpy as np
import pandas as pd
import warnings
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold, KFold, GridSearchCV
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
global scoring
scoring = 'neg_mean_absolute_error'

## functions

In [ ]:
def get_base_model(baseregresor):
    if baseregresor == 'lr':
        return LinearRegression()
    if baseregresor == 'knn':
        return KNeighborsRegressor()
    if baseregresor == 'rfr':
        return RandomForestRegressor()
    if baseregresor == 'svr':
        return LinearSVR(max_iter=100000)

def get_param_grid(clf_method):
    """ given a keyword describing the model type, this
    function returns the parameter grid and the baseline
    model object.
    """
    clf, params = None, []
    if clf_method == 'svr':
        clf = LinearSVR(max_iter=100000)
        params = [{'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
                  'C' : [0.1, 0.5, 1, 5, 10],
                  'degree' : [3,8],
                  'gamma' : ['auto','scale']
                  }]
    
    if clf_method == 'rfr':
        clf = RandomForestRegressor()
        params = [{
                'n_estimators': [100, 150, 200],
                'max_depth': [1, 2, 3],
                'bootstrap': [True],
                'min_samples_split': [5, 10]
                }] 
    if clf_method == 'lr':
        clf = LinearRegression()
        params = None
    if clf_method == 'knn':
        clf = KNeighborsRegressor()
        params = None
    print('estimator:', clf.__class__.__name__)
    return clf, params

def get_best_params(X, y, estimator, paramgrid, nfolds, scoring=scoring):
    """Find the best set of parameter for a given estimator and parameter grid
    using cross validation and grid search.
    """
    cv = KFold(n_splits=nfolds)
    search = GridSearchCV(estimator, paramgrid, scoring=scoring, cv=cv)
    gs = search.fit(X, y)
    best_params = gs.best_params_
    print('best params for %s:'% estimator.__class__.__name__)
    print (best_params)
    return best_params

def get_repeat_cv_scores(X, y, model, nfolds, nrepeats, scoring=scoring):
 
    # define the evaluation procedure
    cv = RepeatedKFold(n_splits=nfolds, n_repeats=nrepeats, random_state=1)
    
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, 
                               X, y, 
                               scoring=scoring, 
                               cv=cv, 
                               n_jobs=-1)
    
    # force the scores to be positive
    n_scores = absolute(n_scores)

    return mean(n_scores), std(n_scores)

def get_direct_multi_output_regression_model(modeltag, params):
    """ Some regression machine learning algorithms support multiple outputs directly.
    This includes most of the popular machine learning algorithms implemented in the
    scikit-learn library, such as: LinearRegression, KNeighborsRegressor,
    DecisionTreeRegressor, RandomForestRegressor. 
    
    Given a modeltag (keyword to identify estimator type), and a set of parameters,
    this wrapper function returns the model object.
    """
    
    if modeltag not in ['lr', 'knn', 'rfr', 'dtr']:
        print('cant do direct multioutputregression for %s' % modeltag)
        return None
    
    # define model
    model = get_base_model(modeltag)
    if params is not None:
        print('params:', params)
        model.set_params(**params)
    
    return model
    
def get_wrapper_multi_output_regression_model(modeltag, params):
    """
    Not all regression algorithms support multioutput regression.
    One example is the support vector regressor.
    A workaround for using regression models designed for predicting one value for 
    multioutput regression is to divide the multioutput regression problem into 
    multiple sub-problems.
    
    Given a modeltag (keyword to identify estimator type), and a set of parameters,
    this wrapper function returns the model object using this approach.
    """
    if modeltag not in ['svr', 'lr', 'knn', 'rfr', 'dtr']:
        print('cant do direct multioutputregression for %s' % modeltag)
        return None
    
    # define model
    model = get_base_model(modeltag)
    if params is not None:
        print('params:', params)
        model.set_params(**params)
    
    wrapper = MultiOutputRegressor(model)
    return wrapper
    
def get_chain_multi_output_regression_model(modeltag, params, order):
    """
    Another approach to using single-output regression models for multioutput regression is
    to create a linear sequence of models.

    The first model in the sequence uses the input and predicts one output; the second model
    uses the input and the output from the first model to make a prediction; and so on.
    
    Given a modeltag (keyword to identify estimator type), and a set of parameters,
    this wrapper function returns the model object using this approach.
    """
    if modeltag not in ['svr', 'lr', 'knn', 'rfr', 'dtr']:
        print('cant do direct multioutputregression for %s' % modeltag)
        return None
    
    # define model
    model = get_base_model(modeltag)
    if params is not None:
        print('params:', params)
        model.set_params(**params)
   
    print('chain order:', order)
    wrapper = RegressorChain(model,order=order)
    return wrapper
    
def standardize_features(df, featurelist):
    """
    force features to have 0 mean and unit std. 
    returns scaled df and the scaler(for use on test set)
    
    """
    print('standardization feature scaling')
    scaler = StandardScaler()
    
    ftrVals = scaler.fit_transform(df)
    out_df = pd.DataFrame(ftrVals, columns=featurelist, index=df.index.values)
   
    return out_df, scaler

def normalize_features(df, featurelist):
    """
    force features to be between 0-1 (aka min-max scaling). 
    returns scaled df and the scaler(for use on test set)
    """
    print('min-max feature scaling')
    scaler = MinMaxScaler()
    ftrVals = scaler.fit_transform(df)
    out_df = pd.DataFrame(ftrVals, columns=featurelist, index=df.index.values)
    return out_df, scaler

## Load the data

### run this cell for loading the data for predicting A-B-C scores

In [ ]:
# comment the line above if you want to run this cell

foldername = 'ABCscores-standardizedftrvals-originaltargetvals'
csvname =  '__.csv'

input_dir = os.getcwd() 
csvpath = os.path.join(input_dir, csvname)
output_dir = os.path.join(input_dir, foldername)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print('input_dir:', input_dir)
print('output_dir:', output_dir, '\n')

df = pd.read_csv(csvpath)
indexcol = 'SampleID'
ftrcols = ['Enhancing','RCBV.Raw_Mean', 'RCBV.Raw_Std', 'FA.Raw_Std', 
           'FA.Raw_Mean', 'MD.Raw_Mean', 'MD.Raw_Std', 'EPI.Raw_Mean', 
           'EPI.Raw_Std', 'CenterFecsT2', 'MeanFecsT2', 'StdFecsT2']

targetcols = ['ScoreA', 'ScoreB', 'ScoreC']

df['Enhancing'].replace(['ENH', 'BAT', 'NEC', 'Central Cyst/Cavity'], [0, 1, 2, 3], inplace=True)
df.set_index(indexcol, inplace=True)
df.dropna(inplace=True)


X = df.loc[:, ftrcols]

if 'standardizedftrvals' in foldername: 
    X, _ = standardize_features(X, ftrcols)
    print(X.head())

elif 'originalftrvals' in foldername:
    print('no feature manipulation')
    
else:
    print('feature preprocessing unknown!! fix this!!')
    X = None # set it to none so the code doesnt just run
    
y = df.loc[:, targetcols]

if 'normalizedtargetvals' in foldername:
    y, _ = normalize_features(y, targetcols)
    print(y.head())
    
elif 'originaltargetvals' in foldername:
    print('no target column manipulation')
    
else:
    print('target column preprocessing unknown!! fix this')
    y = None

print('\n', X.shape, y.shape)

### OR run this cell for loading the data for predicting glMes-glPro-glPN

In [ ]:
foldername = 'GLs-originalftrvals'
csvname =  '__.csv'

run_wrapperregression = False
run_chainregression = False
run_directregression = True

input_dir = os.getcwd() 
csvpath = os.path.join(input_dir, csvname)
output_dir = os.path.join(input_dir, foldername)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print('input_dir:', input_dir)
print('output_dir:', output_dir)

# set index
indexcol = 'SampleID'
df = pd.read_csv(csvpath)
df.set_index(indexcol, inplace=True)

# composite columns
targetcols = ['gl_Mes', 'gl_PN']

print('\ncreating', targetcols)
df['gl_Mes'] = np.NaN
df['gl_PN'] = np.NaN
df['gl_Pro'] = np.NaN
for i in df.index.values:
        
    glmes1 = df.loc[i, 'gl_Mes1']
    glmes2 = df.loc[i, 'gl_Mes2']

    glPro1 = df.loc[i, 'gl_Pro1']
    glPro2 = df.loc[i, 'gl_Pro1']

    glPN1  = df.loc[i, 'gl_PN1']
    glPN2  = df.loc[i, 'gl_PN2']
    
    sum_gls = glmes1 + glmes2 + glPro1 + glPro2 + glPN1 + glPN2
    df.loc[i, 'gl_Mes'] = (glmes1 + glmes2) / sum_gls
    df.loc[i, 'gl_PN'] = (glPN1 + glPN2) / sum_gls
    

# clean up
ftrcols = ['Enhancing','RCBV.Raw_Mean', 'RCBV.Raw_Std', 'FA.Raw_Std', 
           'FA.Raw_Mean', 'MD.Raw_Mean', 'MD.Raw_Std', 'EPI.Raw_Mean', 
           'EPI.Raw_Std', 'CenterFecsT2', 'MeanFecsT2', 'StdFecsT2']

if 'ENH' in set(df['Enhancing'].values):
    df['Enhancing'].replace(['ENH', 'BAT', 'NEC', 'Central Cyst/Cavity'], [0, 1, 2, 3], inplace=True)

df = df[ftrcols+targetcols]
df.dropna(inplace=True)
df.head()

#### separate X and y

In [ ]:
X = df.loc[:, ftrcols]

if 'standardizedftrvals' in foldername: 
    X, _ = standardize_features(X, ftrcols)
    print(X.head())

elif 'originalftrvals' in foldername:
    print('\nno feature manipulation')
    
else:
    print('feature preprocessing unknown!! fix this!!')
    X = None # set it to none so the code doesnt just run
    
y = df.loc[:, targetcols]

print('\n', X.shape, y.shape)

#### visualize target

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
for col in y.columns.values:
    fig = plt.figure(figsize=(2, 3))
    ax = fig.add_subplot(1, 1, 1)
    ax = sns.boxplot(y=col, data=df, palette="Greens")
    plt.show()

## direct multioutputregressionn

In [ ]:
nfolds = 5 
nrepeats = 10

#### get cross validation results

In [ ]:
savedir = os.path.join(output_dir, 'direct_multi_output_regression')
if not os.path.exists(savedir):
    os.makedirs(savedir)

### train cv model
for regtype in ['rfr']:
    
    fold_count = 0
    cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)
    truthcolnames = list(y.columns.values) 
    predcolnames = [v + '_pred' for v in truthcolnames]
    pred_df = pd.DataFrame(index=X.index.values, columns=truthcolnames+predcolnames)
    
    for train_ix, test_ix in cv_outer.split(X, y):

        print('\nfold', fold_count)
        X_train = X.iloc[train_ix]
        X_test  = X.iloc[test_ix]
        y_train = y.iloc[train_ix]
        y_test  = y.iloc[test_ix]
        
        testset_indices = X.index.values[test_ix]
        
        
        # tune 
        estimator, paramgrd = get_param_grid(regtype)
        if paramgrd is not None:
            print('tuning params for %s..' % estimator.__class__.__name__)
            best_params = get_best_params(X, y, estimator, paramgrd, nfolds)
        else:
            best_params = None
        
        model = get_direct_multi_output_regression_model(regtype, best_params)
        mean_s, std_s = get_repeat_cv_scores(X_train, y_train, model, nfolds, nrepeats)
        print('MAE of %s in gridsearch:\t%.3f (%.3f)' % (model.__class__.__name__, mean_s, std_s))


        # train
        model.fit(X_train, y_train)

        # test 
        yhat = model.predict(X_test)

        # save predictions
        pred_df.loc[testset_indices, truthcolnames] = y_test
        pred_df.loc[testset_indices, predcolnames ] = yhat
        fold_count += 1

    print(pred_df.head())
    outputcsvpath = os.path.join(savedir, estimator.__class__.__name__+'_predictions.csv')
    pred_df.to_csv(outputcsvpath)
    print(outputcsvpath)
    

#### create plot of results

In [ ]:
predDir = os.path.join(output_dir, 'direct_multi_output_regression')
figDir =  output_dir
for csv in os.listdir(predDir):
    if '_predictions.csv' in csv:
        
        csvpath = os.path.join(predDir, csv)
        estimator = csv.split('_')[0]
        df = pd.read_csv(csvpath)
        df = df.rename(columns={'Unnamed: 0':indexcol})
        df.set_index(indexcol, inplace=True)
        
        #fig = plt.figure(figsize=(len(scorecols)*4, 3))
        
        for n, col in enumerate(y.columns.values):
            
            #ax = fig.add_subplot( 1, 3, n+1)
            g = sns.lmplot(y=col, x=col+'_pred', data=df)
            r, p = stats.pearsonr(df[col], df[col+'_pred'])
            g.fig.text(.2, .9, 'r={:.2f}, p={:.2g}'.format(r, p), fontsize=14)
            #plt.xlim([0.12,0.6])
            #plt.ylim([0.0,0.8])
            plt.xticks([0.15,0.25,0.35,.45, 0.55],fontsize=15)
            plt.yticks(fontsize=15)
            plt.xlabel('\npredicted glPN')
            plt.ylabel('\nactual glPN')
            plt.show()
            
            figname = '-'.join([s for s in ['direct', estimator, col, 'lmplot.jpg']])
            g.fig.savefig(os.path.join(figDir, figname), bbox_inches='tight', pad_inches=1, dpi=300)

## Wrapper Multioutput Regression Algorithms
hyperparam tuning is different here

#### get cross validation results

In [ ]:
savedir = os.path.join(output_dir, 'wrapper_multi_output_regression')
if not os.path.exists(savedir):
    os.makedirs(savedir)
regtype='svr'
estimator = get_base_model(regtype)
c_grid = [0.1, 0.2, 0.5, 1, 2, 5, 10]

fold_count = 0
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)
truthcolnames = list(y.columns.values) 
predcolnames = [x + '_pred' for x in truthcolnames]
pred_df = pd.DataFrame(index=X.index.values, columns=truthcolnames+predcolnames)

for train_ix, test_ix in cv_outer.split(X, y):

    print('\nfold', fold_count)
    X_train = X.iloc[train_ix]
    X_test  = X.iloc[test_ix]
    y_train = y.iloc[train_ix]
    y_test  = y.iloc[test_ix]
    testset_indices = X.index.values[test_ix]

    # tuning is a bit manual in this case
    # havenot found a way to make it work the traditional way (above)
    scores = []
    for c in c_grid:
        params = {'C':c}
        model = get_wrapper_multi_output_regression_model(regtype, params)
        mean_s, std_s = get_repeat_cv_scores(X, y, model, nfolds, nrepeats)
        scores.append(mean_s)

    best_c = c_grid[scores.index(min(scores))]
    print('\nbest C:', best_c)
    
    params = {'C':best_c}
    model = get_wrapper_multi_output_regression_model(regtype, params)
    mean_s, std_s = get_repeat_cv_scores(X, y, model, nfolds, nrepeats)
    print('MAE of %s in gridsearch:\t%.3f (%.3f)' % (model.__class__.__name__, mean_s, std_s))
    
    # train
    model.fit(X_train, y_train)

    # test 
    yhat = model.predict(X_test)

    # save predictions

    pred_df.loc[testset_indices, truthcolnames] = y_test
    pred_df.loc[testset_indices, predcolnames ] = yhat
    fold_count += 1

print(pred_df.head())
outputcsvpath = os.path.join(savedir, estimator.__class__.__name__+'_predictions.csv')
pred_df.to_csv(outputcsvpath)
print(outputcsvpath)

#### plot results

In [ ]:
predDir = os.path.join(output_dir, 'wrapper_multi_output_regression')
figDir = output_dir
for csv in os.listdir(predDir):
    if '_predictions.csv' in csv:
        
        csvpath = os.path.join(predDir, csv)
        estimator = csv.split('_')[0]
        df = pd.read_csv(csvpath)
        df = df.rename(columns={'Unnamed: 0':indexcol})
        df.set_index(indexcol, inplace=True)
        #fig = plt.figure(figsize=(len(scorecols)*4, 3))
        
        for n, col in enumerate(y.columns.values):
            
            #ax = fig.add_subplot( 1, 3, n+1)
            g = sns.lmplot(y=col, x=col+'_pred', data=df)
            r, p = stats.pearsonr(df[col], df[col+'_pred'])
            g.fig.text(.8, .2, 'r={:.2f}, p={:.2g}'.format(r, p))
            plt.show()
            
            figname = '-'.join([s for s in ['wrapper', estimator, col, 'lmplot.jpg']])
            g.fig.savefig(os.path.join(figDir, figname), bbox_inches='tight', pad_inches=1, dpi=300)

## Chain Multioutput Regression
train chain regression model using support vector regressor

#### get cross validation results

In [ ]:
regtype = 'svr'
orders = [[0,1,2], [0,2,1], [1,0,2], [1,2,0], [2,0,1],[2,1,0]]
scores = []
params = {'C':best_c}
for order in orders:
    model = get_chain_multi_output_regression_model(regtype, params, order)
    ms, _ = get_repeat_cv_scores(X, y, model, nfolds, nrepeats)
    scores.append(ms)
    print(ms)

best_order = orders[scores.index(min(scores))]
print('\n-----best order:',best_order)
print('->'.join(a for a in [targetcols[best_order[0]], 
                            targetcols[best_order[1]], 
                            targetcols[best_order[2]]]))

model = get_chain_multi_output_regression_model(regtype, params, best_order)
_, _ = get_repeat_cv_scores(X, y, model, nfolds, nrepeats)

In [ ]:
savedir = os.path.join(output_dir, 'chain_multi_output_regression')
if not os.path.exists(savedir):
    os.makedirs(savedir)

regtype='svr'
estimator = get_base_model(regtype)

c_grid = [0.1, 0.2, 0.5, 1, 2, 5, 10]
order = best_order
fold_count = 0
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)
truthcolnames = list(y.columns.values) 
predcolnames = [x + '_pred' for x in truthcolnames]
pred_df = pd.DataFrame(index=X.index.values, columns=truthcolnames+predcolnames)

for train_ix, test_ix in cv_outer.split(X, y):

    print('\nfold', fold_count)
    X_train = X.iloc[train_ix]
    X_test  = X.iloc[test_ix]
    y_train = y.iloc[train_ix]
    y_test  = y.iloc[test_ix]
    testset_indices = X.index.values[test_ix]

    # tuning is a bit manual in this case
    # havenot found a way to make it work the traditional way (above)
    scores = []
    for c in c_grid:
        params = {'C':c}
        model = get_chain_multi_output_regression_model(regtype, params, best_order)
        mean_s, std_s = get_repeat_cv_scores(X, y, model, nfolds, nrepeats)
        scores.append(mean_s)

    best_c = c_grid[scores.index(min(scores))]
    print('\nbest C:', best_c)
    
    params = {'C':best_c}
    model = get_wrapper_multi_output_regression_model(regtype, params)
    mean_s, std_s = get_repeat_cv_scores(X, y, model, nfolds, nrepeats)
    print('MAE of %s in gridsearch:\t%.3f (%.3f)' % (model.__class__.__name__, mean_s, std_s))
    
    # train
    model.fit(X_train, y_train)

    # test 
    yhat = model.predict(X_test)

    # save predictions

    pred_df.loc[testset_indices, truthcolnames] = y_test
    pred_df.loc[testset_indices, predcolnames ] = yhat
    fold_count += 1

print(pred_df.head())
outputcsvpath = os.path.join(savedir, estimator.__class__.__name__+'_predictions.csv')
pred_df.to_csv(outputcsvpath)
print(outputcsvpath)

#### plot results

In [ ]:
predDir = os.path.join(output_dir, 'chain_multi_output_regression')
figDir =  output_dir
for csv in os.listdir(predDir):
    if '_predictions.csv' in csv:
        estimatorname= csv.split('_')[0]
        csvpath = os.path.join(predDir, csv)
        estimator = csv.split('_')[0]
        df = pd.read_csv(csvpath)
        df = df.rename(columns={'Unnamed: 0':indexcol})
        df.set_index(indexcol, inplace=True)
        #fig = plt.figure(figsize=(len(scorecols)*4, 3))
        
        for n, col in enumerate(y.columns.values):
            
            #ax = fig.add_subplot( 1, 3, n+1)
            g = sns.lmplot(y=col, x=col+'_pred', data=df)
            r, p = stats.pearsonr(df[col], df[col+'_pred'])
            g.fig.text(.8, .2, 'r={:.2f}, p={:.2g}'.format(r, p))
            plt.title(estimatorname)
            plt.show()
            
            figname = '-'.join([s for s in ['chain', estimatorname, col, 'lmplot.jpg']])
            g.fig.savefig(os.path.join(figDir, figname), bbox_inches='tight', pad_inches=1, dpi=300)

### save direct multioutput random forest regressor model, trained on all data

In [ ]:
print('train and all data, save out the model..')

regtype = 'rfr'
estimator, paramgrd = get_param_grid(regtype)
print('tuning params for %s..' % estimator.__class__.__name__)
best_params = get_best_params(X, y, estimator, paramgrd, nfolds)

model = get_direct_multi_output_regression_model(regtype, best_params)
model.fit(X, y)

# save model to file
modelfullpath = os.path.join(output_dir, 'rfr_' + '-'.join([t for t in targetcols]) + '-wSex.sav')
pickle.dump(model, open(modelfullpath, 'wb'))
print(modelfullpath)